# Quick Clean a Study

This notebook iterates over each BIDS subject in the project, then cleans and saves them as a derivative called `quick_clean`.

In [ ]:
import mne
import mne_bids
import os

In [ ]:
bids_root = '..'
task_name = 'fhbc' # TODO: change this to read from the dataset description
filtering = (1.0, 30.0)

In [ ]:
derivatives_path = '../derivatives/quick_clean'
os.makedirs(derivatives_path, exist_ok=True)

In [ ]:
# Identify all subjects in the dataset
subjects = mne_bids.get_entity_vals(bids_root, entity_key='subject')

# Loop through each subject
for subject in subjects:
    # Read the raw data for the subject
    bids_path = mne_bids.BIDSPath(root=bids_root, subject=subject, task=task_name, suffix='eeg', extension='.edf')
    raw = mne_bids.read_raw_bids(bids_path=bids_path)

    raw.load_data()
    raw = raw.filter(l_freq=filtering[0], h_freq=filtering[1])

    # set up and fit the ICA
    ica = mne.preprocessing.ICA(n_components=20, random_state=97, max_iter=800)
    ica.fit(raw)
    ica.exclude = [1, 2] # details on how we picked these are omitted here
    ica.plot_properties(raw, picks=ica.exclude)
    raw = ica.apply(raw)

    derivative_subject_path = f'{derivatives_path}/sub-{subject}/eeg/' 
    os.makedirs(derivative_subject_path, exist_ok=True)

    raw.save(f'{derivative_subject_path}/sub-{subject}_eeg.fif', overwrite=True)